# Введение в рекомендательные системы

## Коллаборативная фильтрация

В этом задании мы закончим имплементацию коллаборативной фильтрации.

Для этого - выполним действия, необходимые для создания матрицы рейтингов

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook

%matplotlib inline

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)

In [3]:
filepath = 'user_ratedmovies.dat'
df_rates = pd.read_csv(filepath, sep='\t')

In [4]:
filepath = 'movies.dat'
df_movies = pd.read_csv(filepath, sep='\t', encoding='iso-8859-1')

# Перекодируем ID фильмов и пользователей

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc_user = LabelEncoder()
enc_mov = LabelEncoder()

In [7]:
enc_user = enc_user.fit(df_rates.userID.values)
enc_mov = enc_mov.fit(df_rates.movieID.values)

In [8]:
idx = df_movies.loc[:, 'id'].isin(df_rates.movieID)
df_movies = df_movies.loc[idx]

In [9]:
df_rates.loc[:, 'userID'] = enc_user.transform(df_rates.loc[:, 'userID'].values)
df_rates.loc[:, 'movieID'] = enc_mov.transform(df_rates.loc[:, 'movieID'].values)
df_movies.loc[:, 'id'] = enc_mov.transform(df_movies.loc[:, 'id'].values)

In [10]:
df_rates.head()

,userID,movieID,rating,date_day,date_month,date_year,date_hour,date_minute,date_second
0,0,2,1.0,29,10,2006,23,17,16
1,0,31,4.5,29,10,2006,23,23,44
2,0,105,4.0,29,10,2006,23,30,8
3,0,151,2.0,29,10,2006,23,16,52
4,0,154,4.0,29,10,2006,23,29,30


## Матрица рейтингов

In [11]:
from scipy.sparse import coo_matrix, csr_matrix

In [12]:
R = coo_matrix((df_rates.rating.values, (df_rates.userID.values, df_rates.movieID.values)))

In [13]:
R

<2113x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 855598 stored elements in COOrdinate format>

## Похожесть между пользователями

В дальнейшем нам будет удобнее работать с форматом `Compressed Sparse Row matrix`. К счастью переформатировать полученную нами матрицу можно одной командой:

In [14]:
R = R.tocsr()

Теперь, например, рейтинги для первого пользователя можно достать так:

In [15]:
user_1 = R[0]
user_1

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 55 stored elements in Compressed Sparse Row format>

Так как вы возможно не работали с разреженным форматом матриц, устроим небольшой ликбез.

Первым делом, надо понадобится вектор для другого пользователя:

In [16]:
user_2 = R[1]
user_2

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 468 stored elements in Compressed Sparse Row format>

Мы можем сравнивать элементы с 0

In [17]:
user_1_rated = (user_1 != 0)
user_1_rated

<1x10109 sparse matrix of type '<class 'numpy.bool_'>'
	with 55 stored elements in Compressed Sparse Row format>

Можем их "индексировать"

In [18]:
user_1[user_1_rated]

matrix([[1. , 4.5, 4. , 2. , 4. , 4.5, 3.5, 5. , 3.5, 2. , 4. , 3. , 4.5,
         0.5, 4.5, 4. , 3.5, 4.5, 4. , 2.5, 4. , 4. , 4. , 4.5, 2.5, 2. ,
         1.5, 4. , 4. , 4.5, 3. , 3. , 4.5, 3.5, 4.5, 1.5, 3. , 3. , 3.5,
         3.5, 3. , 2.5, 3.5, 4. , 0.5, 4. , 3.5, 4.5, 3.5, 4.5, 5. , 3.5,
         3.5, 3.5, 4.5]])

Можем считать количество ненулевых элементов

In [19]:
user_1.nnz

55

Можем умножать 2 разреженных вектора поэлементно:

In [20]:
(user_1).multiply(user_2)

<1x10109 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>

и скалярно

In [21]:
user_1.dot(user_2.T)[0, 0]

216.75

И превращать разреженную матрицу (вектор) в плотную

In [22]:
user_1_dense = user_1.toarray()
user_1_dense

array([[0., 0., 1., ..., 0., 0., 0.]])

Этого ликбеза вам будет должно быть достаточно, чтобы реализовать функцию расчета похожести между парой пользователей $u$ и $v$:

$$ s_{uv} = \frac{\sum\limits_{i \in I_u\cap I_v} R_{ui} R_{vi}}{\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{ui}^2}}\sqrt{{\sum\limits_{i \in I_u\cap I_v}R_{vi}^2}}}$$

Давайте будем считать, что если количество фильмов которые пользователь $u$ и $v$ посмотрели вместе $<= 2$, то их косинусная мера равна 0.0


### _Решение_

In [23]:
def cosine_similarity_pair_users(u, v):
    u_rated = (u != 0)
    v_rated = (v != 0)
    common_items = (u_rated).multiply(v_rated)
    if common_items.nnz > 2:
        scalar = u.dot(v.T)[0, 0]
        norm = np.linalg.norm(u[common_items]) * np.linalg.norm(v[common_items])
        return scalar / norm
    else:
        return 0.0

## Функция прогнозирования рейтинга

Реализуйте функцию, которая принимает на входе
* Индекс пользователя
* Матрицу рейтингов
* Количество ближайших соседей (несмотря на то, что каждый пользователь - ближайший сосед самому себе, в расчетах он использоваться не должен)

и возвращает вектор с предсказанными рейтингами по всем товарам для этого пользователя

Для того, чтобы считать прогноз по рейтингу мы воспользуемся упрощенной формулой из лекции:

$$ \hat{R}_{ui} = \frac{\sum_{v \in N(u)} s_{uv}R_{vi}}{\sum_{v \in N(u)} \left| s_{uv}\right|} $$


### _Решение_

In [24]:
def rate_items_user(u, R, n_neigbours=30):
    predictions = csr_matrix((1, R.shape[1]))
    cumsim = 0.0
    num_predictions = csr_matrix((1, R.shape[1]))
    s = np.array([cosine_similarity_pair_users(R[u], R[v]) for v in range(R.shape[0])])
    similar_users = np.argsort(s)[::-1]
    print(np.sort(s)[::-1])
    for v in similar_users[:n_neigbours]:
        if v == u:
            continue
        user_sim = s[v]
        predictions += user_sim * R[v]
        cumsim += np.abs(user_sim)
    predictions /= cumsim
    return predictions

В качестве ответа к этому заданию верните 5 идентификаторов фильмов с наивысшим предсказанным рейтингом для пользователя с id 19 (20-я строчка в матрице рейтингов).
* Для усреднения используйте 30 ближайших соседей
* Среди этих 5-и фильмов не должно быть ранее просмотренных фильмов

### _Решение_

In [25]:
R_hat = rate_items_user(20, R, n_neigbours=30)

[1. 1. 1. ... 0. 0. 0.]


In [33]:
rated_items = (R[20] == 0)
unseen_ratings = R_hat.multiply(rated_items)
unseen_ratings = unseen_ratings.toarray()[0]

In [27]:
idx = unseen_ratings.argsort()[::-1]

In [28]:
unseen_ratings[idx]

array([2.5174164 , 2.24147475, 2.15522858, ..., 0.        , 0.        ,
       0.        ])

In [29]:
top5 = idx[:5]

In [30]:
top5

array([2614,  306,  343, 5573, 6720])

In [31]:
assert ', '.join(str(i) for i in top5) == '2614, 306, 343, 5573, 6720'

## Генерация ответа

In [32]:
', '.join(str(i) for i in top5)

'2614, 306, 343, 5573, 6720'